In [1]:
!git clone git@github.com:tejuafonja/margctgan.git
!pip install dython
!pip install pandas==1.5
!pip install scikit-learn==1.0

fatal: destination path 'margctgan' already exists and is not an empty directory.


In [2]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

import os

import sys
# os.chdir(../margctgan)
sys.path.append("./margctgan")

from metrics.marginal import histogram_intersection, column_metric_wrapper, wasserstein_distance, jensonshannon_distance
from metrics.utility import efficacy_test
from metrics.joint import closeness_approximation


from sklearn.metrics import accuracy_score
from functools import partial

In [3]:
def load_data(data_dir, model_name, K, Ep, S=None, C=None, alphas=None):
    dfs = []
    
    if model_name == "VAE":
        for tag in ["dp", "nondp"]:
            if tag == "dp":
                for ep in Ep:
                    for n_iter in [10000]:
                        for k in K:
                            for s in S:
                                for c in C:
                                    fake_csv_path = f"{data_dir}/{model_name}/default/k{k}/fake_{tag}_ep{ep}_C{c}_iter{n_iter}_k{k}_s{s}.csv"
                                    unique_name = f"{model_name}_{tag}_ep{ep}_C{c}_k{k}_s{s}"
                                    df = pd.read_csv(fake_csv_path)

                                    df["ep"]=ep
                                    df["k"]=k
                                    df["s"]=s
                                    df["C"]=c
                                    df["tag"]=tag
                                    df["unique_name"]=unique_name
                                    dfs.append(df)
            elif tag == "nondp":
                for n_iter in [10000]:
                    for k in K:
                        for s in S:
                            if k == 5:
                                fake_csv_path = f"{data_dir}/{model_name}/default/k{k}/fake_{tag}_iter{n_iter}_k{k}_s{s}.csv"
                            else:
                                fake_csv_path = f"{data_dir}/{model_name}/default/k{k}/fake_{tag}_k{k}_s{s}.csv"
                            unique_name = f"{model_name}_{tag}_k{k}_s{s}"
                            df = pd.read_csv(fake_csv_path)
                            df["k"]=k
                            df["s"]=s
                            df["tag"]=tag
                            df["unique_name"]=unique_name
                            dfs.append(df)
    elif model_name == "GAN":
        for tag in ["dp", "nondp"]:
            if tag == "dp":
                for ep in Ep:
                    for n_iter in [10000]:
                        for k in K:
                            for s in S:
                                fake_csv_path = f"{data_dir}/{model_name}/default/k{k}/fake_ep{ep}_k{k}_s{s}.csv"
                                unique_name = f"{model_name}_{tag}_ep{ep}_k{k}_s{s}"
                                df = pd.read_csv(fake_csv_path)

                                df["ep"]=ep
                                df["k"]=k
                                df["s"]=s
                                df["tag"]=tag
                                df["unique_name"]=unique_name
                                dfs.append(df)
            elif tag == "nondp":
                for n_iter in [10000]:
                    for k in K:
                        for s in S:
                            fake_csv_path = f"{data_dir}/{model_name}/default/k{k}/fake_nonpri_k{k}_s{s}.csv"
                            unique_name = f"{model_name}_{tag}_k{k}_s{s}"
                            df = pd.read_csv(fake_csv_path)
                            df["k"]=k
                            df["s"]=s
                            df["tag"]=tag
                            df["unique_name"]=unique_name
                            dfs.append(df)
    elif model_name == "RONGauss":
        for tag in ["dp", "nondp"]:
            if tag == "dp":
                for ep in Ep:
                    for n_iter in [10000]:
                        for k in K:
                            for s in S:
                                fake_csv_path = f"{data_dir}/{model_name}/default/k{k}/ep{ep}_k{k}_s{s}.csv"
                                unique_name = f"{model_name}_{tag}_ep{ep}_k{k}_s{s}"
                                df = pd.read_csv(fake_csv_path)

                                df["ep"]=ep
                                df["k"]=k
                                df["s"]=s
                                df["tag"]=tag
                                df["unique_name"]=unique_name
                                dfs.append(df)
            elif tag == "nondp":
                for n_iter in [10000]:
                    for k in K:
                        for s in S:
                            fake_csv_path = f"{data_dir}/{model_name}/default/k{k}/nonpri_k{k}_s{s}.csv"
                            unique_name = f"{model_name}_{tag}_k{k}_s{s}"
                            df = pd.read_csv(fake_csv_path)
                            df["k"]=k
                            df["s"]=s
                            df["tag"]=tag
                            df["unique_name"]=unique_name
                            dfs.append(df)
                            
    elif model_name == "PGM" or model_name == "Privsyn":
        for tag in ["dp", "nondp"]:
            if tag == "dp":
                for ep in Ep:
                    for n_iter in [10000]:
                        for k in K:
                            for alpha in alphas:
                                fake_csv_path = f"{data_dir}/{model_name}/default/k{k}/fake_ep{ep}_alpha{alpha}_k{k}.csv"
                                unique_name = f"{model_name}_{tag}_ep{ep}_alpha{alpha}_k{k}"
                                df = pd.read_csv(fake_csv_path)

                                df["ep"]=ep
                                df["k"]=k
                                df["alpha"]=alpha
                                df["tag"]=tag
                                df["unique_name"]=unique_name
                                dfs.append(df)
            elif tag == "nondp":
                for n_iter in [10000]:
                    for k in K:
                        for alpha in alphas:
                                fake_csv_path = f"{data_dir}/{model_name}/default/k{k}/fake_nonpri_alpha{alpha}_k{k}.csv"
                                unique_name = f"{model_name}_{tag}_alpha{alpha}_k{k}"
                                df = pd.read_csv(fake_csv_path)

                                df["k"]=k
                                df["alpha"]=alpha
                                df["tag"]=tag
                                df["unique_name"]=unique_name
                                dfs.append(df)


    for k in K:
        # Load train data
        tag="train"
        csv_path = f"{data_dir}/Real Data/default/k{k}_none_filterx.csv"
        unique_name = f"{tag}_k{k}_none_filterx"
        df = pd.read_csv(csv_path)
        df["k"]=k
        df["tag"]=tag
        df["unique_name"]=unique_name
        dfs.append(df)
        
        # Load test data
        tag="test"
        csv_path = f"{data_dir}/Real Data/default/k{k}_none_filterx_test.csv"
        unique_name = f"{tag}_k{k}_none_filterx"
        df = pd.read_csv(csv_path)
        df["k"]=k
        df["tag"]=tag
        df["unique_name"]=unique_name
        dfs.append(df)

    dfs = pd.concat(dfs)                    
    dfs["model_name"]=model_name
    return dfs

def get_param_data(dfs, columns_to_drop=None, **kwargs):
    df = dfs.copy()
    
    for k, v in kwargs.items():
        df = df[df[k] == v]        
            
    return df

def get_score(realdata, 
              fakedata, 
              metric_name, 
              results_dir,
              bins=50,
              neighbors=1,
              unique_name=None):
    
    
    if metric_name == "histogram_intersection":
        cat_cols=["label"]
        fakedata[cat_cols] = fakedata[cat_cols].astype("O")
        realdata[cat_cols] = realdata[cat_cols].astype("O")
        
        metric_func = partial(histogram_intersection, bins=bins)
        score_df = column_metric_wrapper(realdata=realdata, fakedata=fakedata, cat_cols=cat_cols,
                                     column_metric=metric_func)
        score_df.to_csv(f"{results_dir}/{metric_name}_{unique_name}_bin{bins}.csv", index=False)
        score = score_df.score.mean()
    elif metric_name == "wasserstein_distance":
        cat_cols=["label"]
        fakedata[cat_cols] = fakedata[cat_cols].astype("O")
        realdata[cat_cols] = realdata[cat_cols].astype("O")
        
        metric_func = partial(wasserstein_distance, bins=bins)
        score_df = column_metric_wrapper(realdata=realdata, fakedata=fakedata, cat_cols=cat_cols,
                                     column_metric=metric_func)
        score_df.to_csv(f"{results_dir}/{metric_name}_{unique_name}_bin{bins}.csv", index=False)
        score = score_df.score.mean()
    elif metric_name == "jensonshannon_distance":
        cat_cols=["label"]
        fakedata[cat_cols] = fakedata[cat_cols].astype("O")
        realdata[cat_cols] = realdata[cat_cols].astype("O")
        
        metric_func = partial(jensonshannon_distance, bins=bins)
        score_df = column_metric_wrapper(realdata=realdata, fakedata=fakedata, cat_cols=cat_cols,
                                     column_metric=metric_func)
        score_df.to_csv(f"{results_dir}/{metric_name}_{unique_name}_bin{bins}.csv", index=False)
        score = score_df.score.mean()
        
    elif metric_name == "efficacy_test":
        metric_func = efficacy_test
        target_name="label"

        if fakedata.isnull().any().astype("int").sum() != 0:
            # fill na with 0
            # pgm data has nan
            fakedata = fakedata.fillna(0)
        
        realdata[target_name] = realdata[target_name].astype("O")
        fakedata[target_name] = fakedata[target_name].astype("O")
        
        score_df = metric_func(realdata=realdata, fakedata=fakedata, return_dataframe=True,
                                 target_name=target_name, scorer=accuracy_score)
        
        score_df.to_csv(f"{results_dir}/{metric_name}_{unique_name}.csv", index=False)
        score = score_df.score[0] 
        
    elif metric_name == "closeness_approximation" or metric_name == "closeness_approximation_train":     
        cat_cols=["label"]
        
        metric_func = closeness_approximation
        if fakedata.isnull().any().astype("int").sum() != 0:
            # fill na with 0
            # pgm data has nan
            fakedata = fakedata.fillna(0)
        
        fakedata[cat_cols] = fakedata[cat_cols].astype("O")
        realdata[cat_cols] = realdata[cat_cols].astype("O")
        
        
        score_df = metric_func(realdata=realdata, fakedata=fakedata, 
                               neighbors=neighbors, cat_cols=cat_cols, return_mean=False)
        
        score_df = pd.DataFrame(score_df, columns=[i for i in range(1, score_df.shape[1]+1)])
        score_df.to_csv(f"{results_dir}/{metric_name}_{unique_name}_neighbors{neighbors}.csv", index=False)
        
        score = score_df.mean(axis=1).mean()
                
    return score

def add_param_result(dfs, **kwargs):
    df = dfs.copy()
    index = len(dfs)+1
    
    for k, v in kwargs.items():
        df.loc[index, k] = v                
    return df

In [4]:
def compute_result(results_df, testdata, model_name, metric_name, results_dir, columns_to_drop, k, Ep, S=None, C=None, alpha=None, bins=None, 
                   neighbors=None):
    # Compute fake score for private and non-private setting
    for s in S:
        for tag in ["dp", "nondp"]:
            if tag == "dp":
                for ep in Ep:
                    if model_name == "VAE":
                        for c in C:
                            fakedata = get_param_data(dfs, tag=tag, 
                                                 C=c, ep=ep, 
                                                 k=k, s=s)

                            unique_name = fakedata["unique_name"].unique()[0]
                            fakedata = fakedata.drop(columns=columns_to_drop)

                            fakescore = get_score(realdata=testdata, 
                                                  results_dir=results_dir,
                                              fakedata=fakedata, 
                                              bins=bins,
                                              neighbors=neighbors,
                                              metric_name=metric_name, 
                                              unique_name=unique_name)

                            results_df = add_param_result(results_df, tag=tag, k=k, 
                                                      s=s, C=c, ep=ep, bins=bins,
                                                      neighbors=neighbors,
                                                      unique_name=unique_name, 
                                                      model_name=model_name, 
                                                      metric_name=metric_name,
                                                      score=fakescore)
                    
                    elif model_name == "GAN" or model_name == "RONGauss":
                        fakedata = get_param_data(dfs, tag=tag, 
                                             ep=ep, 
                                             k=k, s=s)

                        unique_name = fakedata["unique_name"].unique()[0]
                        fakedata = fakedata.drop(columns=columns_to_drop)

                        fakescore = get_score(realdata=testdata, 
                                              results_dir=results_dir,
                                          fakedata=fakedata, 
                                          bins=bins, neighbors=neighbors,
                                          metric_name=metric_name, 
                                          unique_name=unique_name)

                        results_df = add_param_result(results_df, tag=tag, k=k, 
                                                  s=s, ep=ep, bins=bins,
                                                  neighbors=neighbors,
                                                  unique_name=unique_name, 
                                                  model_name=model_name, 
                                                  metric_name=metric_name,
                                                  score=fakescore)
                        
                    elif model_name == "PGM" or model_name == "Privsyn":
                        for alpha in alphas:
                            fakedata = get_param_data(dfs, tag=tag, alpha=alpha,
                                                 ep=ep, 
                                                 k=k)

                            unique_name = fakedata["unique_name"].unique()[0]
                            fakedata = fakedata.drop(columns=columns_to_drop)

                            fakescore = get_score(realdata=testdata, 
                                                  results_dir=results_dir,
                                              fakedata=fakedata, 
                                              bins=bins, neighbors=neighbors,
                                              metric_name=metric_name, 
                                              unique_name=unique_name)

                            results_df = add_param_result(results_df, tag=tag, k=k, alpha=alpha,
                                                      ep=ep, bins=bins,
                                                      neighbors=neighbors,
                                                      unique_name=unique_name, 
                                                      model_name=model_name, 
                                                      metric_name=metric_name,
                                                      score=fakescore)
                        
            elif tag == "nondp":
                if model_name == "PGM" or model_name == "Privsyn":
                    for alpha in alphas:
                        fakedata = get_param_data(dfs, tag=tag, k=k, alpha=alpha)
                else:
                    fakedata = get_param_data(dfs, tag=tag, k=k, s=s)

                unique_name = fakedata["unique_name"].unique()[0]
                fakedata = fakedata.drop(columns=columns_to_drop)

                fakescore = get_score(realdata=testdata, 
                                      results_dir=results_dir,
                                  fakedata=fakedata, bins=bins,
                                  neighbors=neighbors,
                                  metric_name=metric_name, 
                                  unique_name=unique_name)

                results_df = add_param_result(results_df, tag=tag, 
                                              k=k, s=s, bins=bins,
                                              neighbors=neighbors,
                                              unique_name=unique_name, 
                                              model_name=model_name, 
                                              metric_name=metric_name,
                                              score=fakescore)
    return results_df

In [5]:
def compute_metric_result(dfs, metric_name, model_name, results_dir, K, Ep, neighbors=1, 
                          bins_list=[25, 50, 100], S=None, C=None, alphas=None,
                         ):
    if model_name == "VAE":
        columns_to_drop = ["tag", "ep", "k", "s", "C", "unique_name", "model_name"]
        
    elif model_name == "GAN" or model_name == "RONGauss":
        columns_to_drop = ["tag", "ep", "k", "s", "unique_name", "model_name"]
    
    elif model_name == "PGM" or model_name == "Privsyn":
        columns_to_drop = ["tag", "ep", "k", "alpha", "unique_name", "model_name"]
    
    results_df = pd.DataFrame([], columns=columns_to_drop+["metric_name", "score"])
        
    
    for k in K:
        # Real (train)
        traindata = get_param_data(dfs, tag="train", k=k)
        traindata = traindata.drop(columns=columns_to_drop)

        # Real (fake)
        testdata = get_param_data(dfs, tag="test", k=k)
        testdata = testdata.drop(columns=columns_to_drop)
        
        realscore = get_score(realdata=testdata, results_dir=results_dir,
                  fakedata=traindata, neighbors=neighbors,
                  metric_name=metric_name, 
                  unique_name=f"k{k}_none_filterx")
        
        results_df = add_param_result(results_df, tag="real", k=k,
                                      unique_name=f"k{k}_none_filterx",
                                      model_name="reference", 
                                      metric_name=metric_name,
                                      score=realscore)

        if metric_name == "histogram_intersection" or metric_name=="wasserstein_distance" or metric_name=="jensonshannon_distance":
            for bins in bins_list:
                results_df = compute_result(results_df=results_df, testdata=testdata,model_name=model_name,
                                            results_dir=results_dir,
                                                metric_name=metric_name,
                                                S=S, k=k, C=C, Ep=Ep, 
                                                columns_to_drop=columns_to_drop, bins=bins)
                
        elif metric_name == "efficacy_test":
            results_df = compute_result(results_df=results_df, testdata=testdata,model_name=model_name,
                                        results_dir=results_dir,
                                            metric_name=metric_name,
                                            S=S, k=k, C=C, Ep=Ep, 
                                            columns_to_drop=columns_to_drop)
            
        elif metric_name == "closeness_approximation" or metric_name == "closeness_approximation_train":
            
            
            results_df = compute_result(results_df=results_df, testdata=testdata \
                                        if metric_name == "closeness_approximation" \
                                        else traindata,
                                        model_name=model_name,
                                        results_dir=results_dir,
                                            metric_name=metric_name,
                                            S=S, k=k, C=C, Ep=Ep, neighbors=neighbors,
                                            columns_to_drop=columns_to_drop)
        
        results_df.to_csv(f"{results_dir}/result_df_{metric_name}_{model_name}_k{k}.csv", index=False)
    results_df.to_csv(f"{results_dir}/result_df_{metric_name}_{model_name}.csv", index=False)
    results_df.score = results_df.score.astype("float")
    return results_df

In [6]:
data_dir="../eval_data"
results_dir="./results/"
os.makedirs(results_dir, exist_ok=True)

In [7]:
K=[5, 1000]
S=[1000, 2000]
Ep=[5, 10, 20, 50, 100]

results_df_list = []
dfs_list = []

neighbors=10
bins_list = [25, 50, 100]

metrics = [
           "wasserstein_distance", 
           "jensonshannon_distance",
            "histogram_intersection",
            "efficacy_test", 
            "closeness_approximation", 
            "closeness_approximation_train"
          ]


for model_name in ["VAE", "GAN", "RONGauss", "PGM", "Privsyn"]:    
    if model_name == "VAE":
        C=[0.01]
        alphas=None
        dfs = load_data(data_dir=data_dir, model_name=model_name, K=K, C=C, Ep=Ep, S=S)
        
    elif model_name == "GAN" or model_name == "RONGauss":
        C=None
        alphas=None
        dfs = load_data(data_dir=data_dir, model_name=model_name, K=K, Ep=Ep, S=S)
    
    elif model_name == "PGM" or model_name == "Privsyn":
        C=None
        alphas=[0.25]
        dfs = load_data(data_dir=data_dir, model_name=model_name, K=K, Ep=Ep, alphas=alphas)
     
    dfs_list.append(dfs)
        
    for metric_name in metrics:
        results_df = compute_metric_result(dfs=dfs, 
                                           results_dir=results_dir,
                                           metric_name=metric_name, 
                                           model_name=model_name, 
                                           bins_list=bins_list,
                                           K=K, S=S, C=C, Ep=Ep, 
                                           alphas=alphas, 
                                           neighbors=neighbors)

        results_df_list.append(results_df)

/home/teju/Documents/Projects/PRO-GENE-GEN/eval/stats_eval/./margctgan/metrics/marginal.py:703: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  results = joblib.Parallel(prefer="threads", n_jobs=N_JOBS)(
/home/teju/Documents/Projects/PRO-GENE-GEN/eval/stats_eval/./margctgan/metrics/marginal.py:703: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  results = joblib.Parallel(prefer="threads", n_jobs=N_JOBS)(
/home/teju/Documents/Projects/PRO-GENE-GEN/eval/stats_eval/./margctgan/metrics/marginal.py:703: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  results = joblib.Parallel(prefer="threads", n_jobs=N_JOBS)(
/home/teju/Documents/Projects/PRO-GENE-GEN/eval/stats_eval/./margctgan/metrics/marginal.py:703: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  results = joblib.Parallel(pr

In [8]:
results_df = pd.concat(results_df_list)
print(results_df.shape)
results_df.head()

(1500, 12)


,tag,ep,k,s,C,unique_name,model_name,metric_name,score,bins,neighbors,alpha
1,real,NaN,5,NaN,NaN,k5_none_filterx,reference,wasserstein_distance,0.011726,NaN,NaN,NaN
2,dp,5,5,1000,0.01,VAE_dp_ep5_C0.01_k5_s1000,VAE,wasserstein_distance,0.068287,25.0,NaN,NaN
3,dp,10,5,1000,0.01,VAE_dp_ep10_C0.01_k5_s1000,VAE,wasserstein_distance,0.057748,25.0,NaN,NaN
4,dp,20,5,1000,0.01,VAE_dp_ep20_C0.01_k5_s1000,VAE,wasserstein_distance,0.055133,25.0,NaN,NaN
5,dp,50,5,1000,0.01,VAE_dp_ep50_C0.01_k5_s1000,VAE,wasserstein_distance,0.054887,25.0,NaN,NaN


In [9]:
results_df.to_csv(f"{results_dir}/result_df_all.csv", index=False)